In [1]:
import os
import json
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
import numpy as np
import math
import nltk

# Initial Data Processing of Text in News Articles
In this notebook, I explore the news articles we scraped from the guardian. I have written functions that have the following functionalities:
    - tokenize a string
    - tokenize an article and remove stopwords
    - create doc term matrix
    - create inverted index
    - computes inverse document frequencies (idf values)
    - removes words that do not appear in many documents
    - creates tfidf matrix
    - selects words in a document with the k highest tfidf scores

## Helper Functions

In [ ]:
def tokenize(text, stem_words = False):
    """Returns a list of words that make up the text.
    
    Note: for simplicity, lowercase everything.
    Requirement: Use Regex to satisfy this function
    
    Arguments
    =========
    text: String
    stem_words: boolean
    
    Returns
    =======
    List
    
    """
    stop_words = stopwords.words('english')
    stemmer = PorterStemmer()
    regex = r"([a-zA-Z]+)"
    tokens = re.findall(regex, text.lower())
    if (stem_words):
        return [stemmer.stem(t) for t in tokens if not t in stop_words]
    else:
        return [t for t in tokens if not t in stop_words]

def tokenize_articles(input_data, feature_name, tokenize_method):
    """ adds tokenized transcript (or title) as a feature of each document in dataset
    
    Arguments
    =========
    news_data: list of dictionaries
    
    feature_name: String
    
    tokenize_method: function
    
    Returns 
    =======
    data set with tokenized text for specified feature
    
    """
    for article in input_data:
        new_feature_name = feature_name + '_toks'
        article[new_feature_name] = tokenize_method(article[feature_name])
    return input_data

def create_doc_term_matrix(input_data, feature_name):
    """creates the doc_term matrix
    
    Arguments
    =========
    news_data: list of dictionaries
    
    feature_name: String
    
    Returns
    =======
    list of dictionaries where keys are the types in a document
    corresponding to the index of the list
    
    """
    matrix = []
    for article in input_data:
        tokens = article[feature_name]
        info = {}
        for t in set(tokens):
            info[t] = tokens.count(t)
        info = {k: v for k, v in sorted(info.items(), key=lambda item: item[1], reverse = True)}
        matrix.append(info)
    return matrix

def create_inverted_index(input_doc_term_matrix):
    """creates the inverted index
    
    Arguments
    =========
    input_doc_term_matrix: list of dictionaries
    
    Returns
    =======
    {String : list of tuples}
    
    """
    idx = {}
    doc_id = 0
    for doc in input_doc_term_matrix:
        for key, val in doc.items():
            if idx.get(key) is None:
                idx[key] = []
            idx.get(key).append((doc_id, val))
        doc_id+=1
    return idx

def create_idf(input_inverted_index):
    """computes the inverse document frequency of each type in dataset
    
    Arguments
    =========
    input_inverted_index: {String : list of tuples}
    
    Returns
    =======
    Dictionary {String : Float}
    
    """
    idf = {}
    for key,val in input_inverted_index.items():
        idf[key] = 1/math.log(1+len(val))
    return idf

def create_tfidf_matrix(input_doc_term_matrix, input_idf):
    """creates tf-idf matrix
    
    Arguments
    =========
    input_doc_term_matrix: list of dictionaries
    
    input_idf: {String: Float}
    
    Returns
    =======
    list of dictionaries
    
    """
    matrix = []
    for doc in input_doc_term_matrix:
        info = {}
        for key,tf in doc.items():
            idf = 0
            if (input_idf.get(key) is not None):
                idf = input_idf.get(key)
            info[key] = tf*idf
        info = {k: v for k, v in sorted(info.items(), key=lambda item: item[1], reverse = True)}
        matrix.append(info)
    return matrix

def remove_non_frequent_types(input_inverted_index, min_df):
    """Removes terms from inverted index 
    that appear in less than min_df documents
    
    Arguments
    =========
    input_inverted_index: {String : list of tuples}
    
    min_df: Int
    
    Returns
    =======
    {String : List of Tuples}
    
    """
    total_types = len(input_inverted_index)
    idx_copy = input_inverted_index.copy()
    count = 0
    for key,val in idx_copy.items():
        if (len(val) < min_df):
            del input_inverted_index[key]
            count+=1
    print("Percent of Types Removed: ", count/(total_types*1.0))
    print("Number of Types Remaining: ", total_types - count)
    return input_inverted_index

def compute_doc_norms(input_tfidf_matrix):
    """ computes l2 norm of each document
    
    Arguments
    ========
    input_tfidf_matrix: List of Dictionaries
    
    Returns
    =======
    numpy.array
    
    """
    norms = []
    doc_id = 0
    for doc in input_tfidf_matrix:
        accum = 0
        for val in doc.values():
            accum += math.pow(val, 2)
        norms.append(math.sqrt(accum))
        doc_id += 1
    return norms


def run(feature, input_data, min_df=0):
    # Create doc term matrix
    doc_term_matrix = create_doc_term_matrix(input_data, feature + '_toks')
    # Create inverted index
    inverted_index = create_inverted_index(doc_term_matrix)
    inverted_index = remove_non_frequent_types(inverted_index, min_df)
    # Compute idf values
    idf_values = create_idf(inverted_index)
    # Create tfidf matrix
    tfidf_matrix = create_tfidf_matrix(doc_term_matrix, idf_values)
    # Compute doc norms
    norms = compute_doc_norms(tfidf_matrix)
    # Save data structures
    
    #Camilo's Changes
    save_json_file(feature + '_idf_values.json', idf_values)
    save_json_file(feature + '_inverted_index.json', inverted_index)
    save_json_file(feature + '_tfidf_matrix.json', tfidf_matrix)
    save_json_file(feature + '_norms.json', norms)
    
    #Ein's changes
    """save_json_file(feature + '_inverted_index1.json', inverted_index)
    save_json_file(feature + '_tfidf_matrix1.json', tfidf_matrix)
    save_json_file(feature + '_norms1.json', norms)"""

$$idf(i) = \frac{1}{1 + log(df(i))}$$

In [29]:
def save_as_json(file_name, input_data):
    with open(name, 'w') as f:
        f.write(json.dumps(input_data, ensure_ascii=False,indent=2))

## Main Method Begins in the Cell Below

In [30]:
with open('final_data_url.json', encoding = 'utf-8') as json_file:
    data = json.load(json_file)

# Concurrency Matrix

In [69]:
def boolean_search_and(term_A, term_B, idx):
    i = 0
    j = 0
    list_A = idx[term_A]
    list_B = idx[term_B]
    score = 0
    while((i < len(list_A)) & (j < len(list_B))):
        doc_A = list_A[i][0]
        df_A = list_A[i][1]
        doc_B = list_B[j][0]
        df_B = list_B[j][1]
        if (doc_A == doc_B):
            score += 1
            i+=1
            j+=1
        elif (doc_A < doc_B):
            i+=1
        else:
            j+=1
    return score
            
        

In [124]:
concurrent = {}
keys = list(transcript_inverted_index.keys())
doc_freq = [len(transcript_inverted_index[k]) for k in keys]    

In [125]:
for i in range(len(keys)):
    key1 = keys[i]
    concurrent[key1] = []
    for j in range(i):
        key2 = keys[j]
        score = boolean_search_and(key1, key2, transcript_inverted_index)
        if (score > 0):
            concurrent[key1].append((key2, score/(doc_freq[j]*doc_freq[i])))
            concurrent[key2].append((key1, score/(doc_freq[j]*doc_freq[i])))
    #scores.sort(key = lambda x: x[1], reverse = True)
    #concurrent[key1] = scores[:20]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 